You are given a 0-indexed array of positive integers nums and a positive integer limit.

In one operation, you can choose any two indices i and j and swap nums[i] and nums[j] if |nums[i] - nums[j]| <= limit.

Return the lexicographically smallest array that can be obtained by performing the operation any number of times.

An array a is lexicographically smaller than an array b if in the first position where a and b differ, array a has an element that is less than the corresponding element in b. For example, the array [2,10,3] is lexicographically smaller than the array [10,2,3] because they differ at index 0 and 2 < 10.

 

Example 1:

Input: nums = [1,5,3,9,8], limit = 2
Output: [1,3,5,8,9]
Explanation: Apply the operation 2 times:
- Swap nums[1] with nums[2]. The array becomes [1,3,5,9,8]
- Swap nums[3] with nums[4]. The array becomes [1,3,5,8,9]
We cannot obtain a lexicographically smaller array by applying any more operations.
Note that it may be possible to get the same result by doing different operations.
Example 2:

Input: nums = [1,7,6,18,2,1], limit = 3
Output: [1,6,7,18,1,2]
Explanation: Apply the operation 3 times:
- Swap nums[1] with nums[2]. The array becomes [1,6,7,18,2,1]
- Swap nums[0] with nums[4]. The array becomes [2,6,7,18,1,1]
- Swap nums[0] with nums[5]. The array becomes [1,6,7,18,1,2]
We cannot obtain a lexicographically smaller array by applying any more operations.
Example 3:

Input: nums = [1,7,28,19,10], limit = 3
Output: [1,7,28,19,10]
Explanation: [1,7,28,19,10] is the lexicographically smallest array we can obtain because we cannot apply the operation on any two indices.
 

Constraints:

1 <= nums.length <= 105
1 <= nums[i] <= 109
1 <= limit <= 109

In [1]:
# union find topic의 문제라는데, 과연 이게 무엇일까


1

In [3]:
test = [1,2,5,6,1]
test[0],test[1] = test[1],test[0]
test

[2, 1, 5, 6, 1]

In [ ]:
이 문제는 실제로 Union-Find (Disjoint Set Union, DSU) 알고리즘으로 해결할 수 있습니다. 핵심 아이디어는 다음과 같습니다:

숫자 간의 swap 조건인 |nums[i] - nums[j]| <= limit 를 만족하는 경우, 두 인덱스 i와 j를 같은 그룹으로 묶습니다.
그룹 안에서 숫자들을 정렬하여 각 인덱스에 알맞은 값을 배치합니다.
결과적으로, 같은 그룹 내에서만 숫자들이 교환 가능하므로 lexicographically smallest array를 얻을 수 있습니다.
알고리즘
Union-Find 데이터 구조를 사용하여 서로 swap 가능한 인덱스들을 그룹화합니다.
각 그룹을 정렬하여 해당 그룹 내의 인덱스들에 lexicographically smallest 배열을 만듭니다.
구현 코드
다음은 Python 코드입니다:

In [ ]:
from typing import List
from collections import defaultdict

class Solution:
    def lexicographicallySmallestArray(self, nums: List[int], limit: int) -> List[int]:
        n = len(nums)
        parent = list(range(n))  # Union-Find parent array
        
        # Find function with path compression
        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])
            return parent[x]
        
        # Union function
        def union(x, y):
            rootX = find(x)
            rootY = find(y)
            if rootX != rootY:
                parent[rootX] = rootY
        
        # Union-Find to group indices based on the condition
        for i in range(n):
            for j in range(i + 1, n):
                if abs(nums[i] - nums[j]) <= limit:
                    union(i, j)
        
        # Group indices by their root parent
        groups = defaultdict(list)
        for i in range(n):
            groups[find(i)].append(i)
        
        # Sort each group and rearrange nums
        result = nums[:]
        for indices in groups.values():
            values = sorted(result[i] for i in indices)  # Sort values in the group
            indices.sort()  # Sort indices for in-place replacement
            for idx, val in zip(indices, values):
                result[idx] = val
        
        return result
# 그치만 이 풀이도 time limit에 걸린다

In [ ]:
# 이게 시간 빠른 풀이..
class Solution:
    def lexicographicallySmallestArray(self, nums: List[int], limit: int) -> List[int]:
        n = len(nums)
        # Pair each number with its index and sort by the number
        sorted_enum = sorted((num, i) for i, num in enumerate(nums))
        
        new_positions = []
        curr_positions = []
        prev = float('-inf')
        
        for num, idx in sorted_enum:
            # If the current number exceeds the previous number by more than the limit,
            # sort and append the current positions to the result
            if num > prev + limit:
                new_positions.extend(sorted(curr_positions))
                curr_positions = [idx]
            else:
                curr_positions.append(idx)
            prev = num
        
        # Append any remaining positions
        new_positions.extend(sorted(curr_positions))
        
        # Construct the result array using the new positions
        res = [0] * n
        for i, idx in enumerate(new_positions):
            res[idx] = sorted_enum[i][0]
        
        return res

In [ ]:
# C++
class Solution {
public:
    vector<int> lexicographicallySmallestArray(vector<int>& nums, int limit) {
        int n = nums.size();
        vector<int> map;
        vector<pair<int, int>> pairs;
        for (int i = 0; i < n; ++i) {
            pairs.push_back({nums[i], i});
            map.push_back(0);
        }
        sort(pairs.begin(), pairs.end(), [&](auto& a, auto& b) { return a.first < b.first; });
        vector<pair<int, int>> sets = {{n - 1, n - 1}};
        for (int i = n - 2, group = 0; i >= 0; --i) {
            if (pairs[i + 1].first - pairs[i].first <= limit) {
                sets.back().first = i;
                map[pairs[i].second] = group;
            } 
            else {
                sets.push_back({i, i});
                map[pairs[i].second] = ++group;
            }
        }
        for (int i = 0; i < n; ++i) {
            nums[i] = pairs[sets[map[i]].first++].first;
        }
        return nums;
    }
};

In [ ]:
# C
/**
 * Note: The returned array must be malloced, assume caller calls free().
 */

// cmp function to sort copy of original array
int cmp(const void *a, const void *b){
    int *arr1 = (int *)a;
    int *arr2 = (int *)b;
    return (arr1[0] - arr2[0]);
}

// cmp function to sort indices of the cpy array
int cmp2(const void *a, const void *b){
    return *(int *)a - *(int *)b;
}

int* lexicographicallySmallestArray(int* nums, int numsSize, int limit, int* returnSize) {
    int *ret = (int *)malloc(sizeof(int) * numsSize);
    *returnSize = numsSize;
    int cpy[numsSize][2]; // cpy[i][0] is the value, cpy[i][1] is its original pos/index in the array

    // ex: {1, 7, 6, 18, 2, 1} -> {{1,0},{7,1},{6,2},{18,3},{2,4},{1,5}}
    for (int i = 0; i < numsSize; i++){
        cpy[i][0] = nums[i];
        cpy[i][1] = i;
    }

    qsort(cpy, numsSize, sizeof(cpy[0]), cmp); // sort cpy by its values
    // following the example above
    // {{1,0},{7,1},{6,2},{18,3},{2,4},{1,5}} ->
    // {{1,0},{1,5},{2,4},{6,2},{7,1},{18,3}}

    int indices[numsSize]; // array to store the indices of the original values
    for (int i = 0; i < numsSize;){
        int size = 0;         // size of the current group
        int prev = cpy[i][0]; // track the value of the last value in the group
        // loop through cpy until the next value is too high to be apart of the same group
        // ex: limit = 3 cpy[i][1] = {1, 1, 2, 6, 7, 18}; first loop here it would stop at i = 3 because (6 - 3) > 2;
        while(i < numsSize && cpy[i][0] <= (prev + limit)){
            indices[size++] = cpy[i][1];
            prev = cpy[i][0];
            i++;
        }
        // ex: indices = {0,5,4}
        // sort the indices of values in the same group
        qsort(indices, size, sizeof(int), cmp2);
        // indices: {0,4,5}
        // assign values of cpy in accordance to the sorted indices array
        // k - i is to start at the last index copied from the cpy array
        for (int j = 0, k = i - size; j < size; j++, k++){
            ret[indices[j]] = cpy[k][0];
            // ret[0] = 1
            // ret[4] = 1
            // ret[5] = 2
            // ret = {1, ?, ?, ?, 1, 2};
        }
    }

    return ret;
}